In [1]:
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('2014_to_march2020_btc_minutes_data')
df.shape

(2602679, 6)

In [4]:
df.head()

,time,open,close,high,low,volume
0,2014-01-01 08:01:00,746.99,747.00,747.00,746.99,2.000000
1,2014-01-01 08:02:00,743.89,744.89,744.89,743.89,0.020695
2,2014-01-01 08:04:00,745.01,745.00,745.01,745.00,0.900000
3,2014-01-01 08:05:00,747.00,747.30,747.30,747.00,2.500000
4,2014-01-01 08:07:00,744.87,744.87,744.87,744.87,1.230769


In [15]:
def labelling(df, col_idx, window=11):
    """
    Takes a pandas dataframe, sliding window size (default=11), and column index for 'close'
    
    GOAL: create a slidding window, check the middle value of the window, if its the window min() = BUY
    if its the window max() = SELL, otherwise = HOLD. Over the time period of the window find local peaks
    and troughs which would be "BUY" and "SELL" signals.
    
    df: Pandas DataFrame, should have open, close, high, low, and volumn but must have close
    window: int, number of periods used to create the sliding window
    col_idx: int, index number of 'close' column
    
    return: Pandas DataFrame with new column 'labels' which has "BUY", "SELL", and "HOLD" values
    """
    
    # set up the sliding window, mid point, column index, and end value of for loop
    period = int(window)
    mid_point = int((period)/ 2)
    close_col = int(col_idx)
    end = len(df)

    # get start time and print out starting message
    start_time = datetime.now()
    print("starting to label the data")

    # loop through dataframe
    for i in range(end):
        # if window would extend out of index of dataframe pass
        if (i + period) > end:
            pass
        
        else:
            # create window, locate closing price at mid point index
            window = df.iloc[i:i+period, close_col]
            mid_price = df.iat[i+mid_point, close_col]
            
            # use window and mid point to update 'label' column with correct value
            if window.min() == mid_price:
                df.at[i+mid_point, 'label'] = "BUY"
            elif window.max() == mid_price:
                df.at[i+mid_point, 'label'] = "SELL"
            else:
                df.at[i+mid_point, 'label'] = "HOLD"
                
    # get ending time and print out duration
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    # return dataframe with labels
    return df

In [16]:
df = labelling(df, 2, 11)
df.head(10)

starting to label the data
Duration: 0:07:14.366638


,time,open,close,high,low,volume,label
0,2014-01-01 08:01:00,746.99,747.00,747.00,746.99,2.000000,NaN
1,2014-01-01 08:02:00,743.89,744.89,744.89,743.89,0.020695,NaN
2,2014-01-01 08:04:00,745.01,745.00,745.01,745.00,0.900000,NaN
3,2014-01-01 08:05:00,747.00,747.30,747.30,747.00,2.500000,NaN
4,2014-01-01 08:07:00,744.87,744.87,744.87,744.87,1.230769,NaN
5,2014-01-01 08:08:00,744.84,744.87,744.87,744.84,0.371690,HOLD
6,2014-01-01 08:10:00,744.84,744.84,744.84,744.84,0.090297,HOLD
7,2014-01-01 08:12:00,744.00,744.50,744.50,744.00,4.963859,HOLD
8,2014-01-01 08:13:00,744.84,744.50,744.84,744.17,0.959867,HOLD
9,2014-01-01 08:14:00,744.17,744.17,744.17,744.17,0.047933,BUY


In [21]:
df.tail(10)

,time,open,close,high,low,volume,label
2602669,2020-03-26 02:02:00,6703.900000,6703.300000,6704.000000,6701.8,1.072065,HOLD
2602670,2020-03-26 02:03:00,6703.200000,6706.900000,6707.000000,6703.2,0.029335,SELL
2602671,2020-03-26 02:04:00,6707.000000,6703.980099,6707.000000,6703.8,0.625943,HOLD
2602672,2020-03-26 02:05:00,6704.100000,6702.800000,6707.500000,6701.3,0.545650,HOLD
2602673,2020-03-26 02:06:00,6705.400000,6700.400000,6705.400000,6700.1,0.047511,HOLD
2602674,2020-03-26 02:07:00,6700.400000,6700.000000,6700.400000,6700.0,0.440617,NaN
2602675,2020-03-26 02:08:00,6700.000000,6681.500000,6700.008115,6675.1,20.049320,NaN
2602676,2020-03-26 02:09:00,6681.400000,6677.900000,6681.400000,6676.1,0.502999,NaN
2602677,2020-03-26 02:10:00,6679.800000,6679.700000,6680.900000,6676.0,0.821444,NaN
2602678,2020-03-26 02:11:00,6679.754384,6677.500000,6679.800000,6677.5,0.693173,NaN


In [22]:
df['label'].value_counts()

HOLD    2118174
SELL     242851
BUY      241644
NaN          10
Name: label, dtype: int64

In [23]:
df.to_csv('btc_minutes_data_labeled')